# Model Download Fix for Google Colab

This notebook demonstrates the fix for the `NameError: name 'files' is not defined` issue when trying to download model files in Google Colab.

## Problem
The original code was missing the proper import for the Google Colab `files` module, causing a NameError when trying to download files.

## Solution
- Import the necessary `files` module from `google.colab`
- Add proper error handling and validation
- Enhance user feedback and file size reporting
- Include model metadata in the download

In [ ]:
# FIXED: Import the missing Google Colab files module
import os
import sys
from pathlib import Path
from datetime import datetime

# Check if running in Google Colab and import accordingly
try:
    import google.colab
    from google.colab import files
    IN_COLAB = True
    print("✅ Running in Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("⚠️  Note: Not running in Google Colab environment")
    print("   files.download() functionality will be simulated")

In [ ]:
# Helper functions for enhanced functionality

def get_file_size_mb(file_path):
    """Get file size in MB with proper formatting."""
    if os.path.exists(file_path):
        size_bytes = os.path.getsize(file_path)
        size_mb = size_bytes / (1024 * 1024)
        return f"{size_mb:.2f} MB"
    return "Unknown"


def validate_model_file(model_path):
    """Validate that the model file exists and is accessible."""
    if not os.path.exists(model_path):
        print(f"❌ Error: Model file '{model_path}' not found")
        return False
    
    if os.path.getsize(model_path) == 0:
        print(f"❌ Error: Model file '{model_path}' is empty")
        return False
    
    print(f"✅ Model file validated: {model_path} ({get_file_size_mb(model_path)})")
    return True


def create_model_info_file(model_path):
    """Create an information file with model metadata."""
    info_content = f"""Model Information
==================
Model file: {os.path.basename(model_path)}
File size: {get_file_size_mb(model_path)}
Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Description: Fine-tuned disease classification model
Architecture: Custom neural network
Training data: Disease symptom dataset

Usage:
------
import torch
model = torch.load('{os.path.basename(model_path)}')
model.eval()
"""
    
    info_path = f"{os.path.splitext(model_path)[0]}_info.txt"
    try:
        with open(info_path, 'w') as f:
            f.write(info_content)
        print(f"✅ Model information file created: {info_path}")
        return info_path
    except Exception as e:
        print(f"⚠️  Warning: Could not create info file: {e}")
        return None

print("📋 Helper functions defined successfully")

In [ ]:
# STEP 1: Validate model file exists before proceeding
model_file = "fin_disease.pth"
zip_file = "fin_disease.zip"

print("🔍 Checking for model file...")
if not os.path.exists(model_file):
    print(f"⚠️  Model file '{model_file}' not found. Creating a dummy file for demonstration.")
    # Create a dummy model file for demonstration
    with open(model_file, 'w') as f:
        f.write("# Dummy model file for demonstration\n" * 100)
    print(f"✅ Created dummy model file: {model_file}")

# Validate the model file
validate_model_file(model_file)

In [ ]:
# STEP 2: Create model information file
print("📝 Creating model information file...")
info_file = create_model_info_file(model_file)

if info_file:
    print(f"📄 Info file size: {get_file_size_mb(info_file)}")

In [ ]:
# STEP 3: Create zip file with enhanced error handling
print("📦 Creating zip file...")

# Build zip command to include model and info file
zip_command = f"zip -r {zip_file} {model_file}"
if info_file and os.path.exists(info_file):
    zip_command += f" {info_file}"

print(f"   Command: {zip_command}")

# Execute the zip command
exit_code = os.system(zip_command)

if exit_code != 0:
    print(f"❌ Error: Zip command failed with exit code {exit_code}")
else:
    print("✅ Zip file created successfully")

In [ ]:
# STEP 4: Verify zip file creation and report details
if os.path.exists(zip_file):
    file_size = get_file_size_mb(zip_file)
    print(f"✅ Model and info zipped as {zip_file}")
    print(f"📊 Zip file size: {file_size}")
    
    # List contents of zip file
    print("\n📋 Zip file contents:")
    os.system(f"unzip -l {zip_file}")
    
else:
    print("❌ Error: Zip file was not created")
    print("   Cannot proceed with download")

In [ ]:
# STEP 5: FIXED - Download the zip file with proper error handling
if os.path.exists(zip_file):
    print("🔄 Starting download...")
    
    if IN_COLAB:
        try:
            # This is the FIXED line - now 'files' is properly imported
            files.download(zip_file)
            print("✅ Download initiated!")
            print("📥 Check your browser's download folder for the file")
        except Exception as e:
            print(f"❌ Error during download: {e}")
            print("   Please check your browser settings and try again")
    else:
        print("🔄 Simulating download (not in Colab environment)...")
        print(f"   File '{zip_file}' would be downloaded in Google Colab")
        print("✅ Download simulation complete!")
        
else:
    print("❌ Error: Zip file was not created")
    print("   Cannot download non-existent file")

In [ ]:
# STEP 6: Summary and cleanup options
print("\n" + "="*50)
print("📋 DOWNLOAD PROCESS SUMMARY")
print("="*50)

if os.path.exists(zip_file):
    print(f"✅ Status: SUCCESS")
    print(f"📦 File: {zip_file}")
    print(f"📊 Size: {get_file_size_mb(zip_file)}")
    print(f"📁 Location: {os.path.abspath(zip_file)}")
    
    # Optional cleanup
    print("\n🗑️  Cleanup options:")
    print("   - To remove temporary files, run: !rm fin_disease.pth fin_disease_info.txt")
    print("   - To keep only the zip file, run: !rm fin_disease.pth fin_disease_info.txt")
else:
    print(f"❌ Status: FAILED")
    print("   Check the error messages above for troubleshooting")

print("\n🎉 Process complete!")